In [6]:
import jax
import jax.numpy as jnp
from flax import linen as nn
import optax

import tensorflow_datasets as tfds


In [7]:
# MNIST 데이터 불러오기
ds = tfds.load('mnist', split='train', as_supervised=True)

In [8]:
# 데이터 전처리 함수
def preprocess(image, label):
    image = jnp.array(image, dtype=jnp.float32) / 255.0
    label = jnp.array(label, dtype=jnp.int32)
    return image.reshape(-1), label

# 데이터 변환
train_data = [(preprocess(image, label)) for image, label in tfds.as_numpy(ds)]
print(f'훈련 데이터 샘플 수: {len(train_data)}')

훈련 데이터 샘플 수: 60000


In [9]:
# 신경망 모델 정의 (Flax 사용)

class MLP(nn.Module):
    hidden_dim: int
    output_dim: int = 10

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(self.hidden_dim)(x)
        x = nn.relu(x)
        x = nn.Dense(self.output_dim)(x)
        return x


In [10]:
# 모델 초기화
key = jax.random.PRNGKey(42)
model = MLP(hidden_dim=128, output_dim=10)

# 더미 입력으로 파라미터 초기화
dummy_x = jnp.ones((1, 784))
params = model.init(key, dummy_x)


In [11]:
# Optimizer 정의 (Optax 사용)
learning_rate = 0.01
optimizer = optax.adam(learning_rate)
opt_state = optimizer.init(params)


In [12]:
# 손실함수 및 정확도 계산 (Optax 사용)

## 손실함수
def loss_fn(params, x, y):
    logits = model.apply(params, x)
    loss = optax.softmax_cross_entropy_with_integer_labels(
        logits, y
    ).mean()
    return loss

## 정확도 계산 함수
def accuracy_fn(params, x, y):
    logits = model.apply(params, x)
    predictions = jnp.argmax(logits, axis=1)
    return jnp.mean(predictions == y)


In [13]:
# 학습 루프 정의

## 훈련단계
epochs = 5
batch_size = 128

# 파라미터 업데이트 함수 (JIT 컴파일)
@jax.jit
def update_step(params, opt_state, x, y):
    loss, grads = jax.value_and_grad(loss_fn)(params, x, y)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    return params, opt_state, loss


In [14]:
# 모델 학습

for epoch in range(epochs):
    # 미니배치 학습
    for i in range(0, len(train_data), batch_size):
        batch = train_data[i:i + batch_size]
        x_batch, y_batch = zip(*batch)
        x_batch = jnp.stack(x_batch)
        y_batch = jnp.array(y_batch)

        # 파라미터 업데이트
        params, opt_state, loss = update_step(params, opt_state, x_batch, y_batch)

    # 에포크별 손실 및 정확도 출력
    train_acc = accuracy_fn(params, x_batch, y_batch)
    print(f"Epoch {epoch + 1}, Loss: {loss:.4f}, Accuracy: {train_acc:.4f}")


Epoch 1, Loss: 0.0697, Accuracy: 0.9896
Epoch 2, Loss: 0.0392, Accuracy: 0.9896
Epoch 3, Loss: 0.0302, Accuracy: 1.0000
Epoch 4, Loss: 0.0144, Accuracy: 1.0000
Epoch 5, Loss: 0.0502, Accuracy: 1.0000


In [15]:
# 학습 결과 평가

# 테스트 데이터 로드
ds_test = tfds.load('mnist', split='test', as_supervised=True)
test_data = [(preprocess(image, label)) for image, label in tfds.as_numpy(ds_test)]

# 평가
x_test, y_test = zip(*test_data)
x_test = jnp.stack(x_test)
y_test = jnp.array(y_test)

test_acc = accuracy_fn(params, x_test, y_test)
print(f"테스트 정확도: {test_acc:.4f}")


테스트 정확도: 0.9714
